In [1]:
import os
import csv
train_file=os.path.join('/home/wzm/test/rumor_detect/AAAI_dataset', 'politi_train.csv')
test_file=os.path.join('/home/wzm/test/rumor_detect/AAAI_dataset', 'politi_test.csv')
def read_csv(file):
    dataset=[]
    with open('{}'.format(file), encoding="utf-8") as f:
        reader = csv.reader(f)
        for idx,row in enumerate(reader):
            if idx >= 1:
                text = row[1]
                img_name=row[2]
                label=row[3]
                if label == '0':
                    dataset.append((text, img_name, 0))
                elif label=='1':
                    dataset.append((text, img_name, 1))
    return dataset
train_dataset=read_csv(train_file)
test_dataset=read_csv(test_file)

In [2]:
len(train_dataset),len(test_dataset)


(381, 104)

In [3]:
label_set=[[],[]]
for idx, data in enumerate(train_dataset):
    _,_,label=data
    if label==0:
        label_set[0].append(idx)
    else:
        label_set[1].append(idx)
print(label_set)
print(len(label_set[0]))
print(len(label_set[1]))

[[0, 1, 10, 14, 24, 27, 30, 33, 36, 38, 39, 48, 49, 62, 63, 65, 70, 72, 75, 76, 77, 78, 79, 81, 83, 84, 86, 91, 93, 96, 101, 108, 109, 110, 114, 116, 117, 118, 122, 123, 128, 131, 132, 135, 137, 138, 139, 141, 142, 143, 147, 151, 154, 156, 157, 165, 166, 168, 169, 176, 178, 179, 185, 186, 187, 188, 189, 190, 193, 194, 196, 197, 200, 201, 203, 204, 206, 211, 212, 215, 220, 223, 225, 226, 228, 231, 238, 241, 242, 250, 251, 253, 256, 258, 261, 268, 269, 271, 277, 278, 279, 286, 287, 292, 293, 298, 301, 303, 304, 305, 312, 318, 319, 323, 324, 325, 337, 338, 341, 343, 344, 345, 346, 351, 353, 356, 361, 363, 364, 367, 369, 370, 372, 373, 377], [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 29, 31, 32, 34, 35, 37, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64, 66, 67, 68, 69, 71, 73, 74, 80, 82, 85, 87, 88, 89, 90, 92, 94, 95, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 111, 112, 113, 115, 119, 120, 121, 124, 125, 1

In [4]:
train_dataset[0]

('Democrats go full TYRANNY: Now demand nationwide gun confiscation from law-abiding Americans… at gunpoint, of course by: Ethan Huff\n\nNaturalNews.com\n\nMonday, May 07, 2018\n\nA California Democrat was recently given a platform by USA Today to publish a shocking editorial that calls for nationwide confiscation of all “military-style semiautomatic assault weapons” from law-abiding citizens.\n\nRepresentative Eric Swalwell from California’s 15th District wrote that all so-called “assault” weapons need to be banned, and that a federal gun buy-back program needs to be instituted in order to effectively collect them all from the citizenry.\n\nRep. Swalwell even goes a step further, insisting that those who refuse to hand over their “assault” weapons be criminally prosecuted – including law-abiding gun owners who have never been convicted of committing a crime with their legally-purchased weaponry.\n\nNot content to simply impose a fresh ban on all new “assault” weapon purchases, which i

In [5]:
test_dataset[0]

('Philando Castile Had Been Stopped 52 Times By Police – WCCO \n\n\n\n— When Philando Castile saw the flashing lights in his rearview mirror the night he got shot, it wasn’t unusual. He had been pulled over at least 52 times in recent years in and around the Twin Cities and given citations for minor offenses including speeding, driving without a muffler and not wearing a seat belt.\n\nHe was assessed at least $6,588 in fines and fees, although more than half of the total 86 violations were dismissed, court records show.\n\nWas Castile an especially bad driver or just unlucky? Or was he targeted by officers who single out black motorists like him for such stops, as several of his family members have alleged?\n\nThe answer may never be known, but Castile’s stop for a broken tail light Wednesday ended with him fatally shot by a suburban St. Paul police officer, and Castile’s girlfriend livestreaming the chilling aftermath.\n\nThe shooting has added a new impetus to a national debate on ra

In [6]:
import nltk
import torch
import torch.nn as nn
from transformers import XLNetTokenizer, XLNetModel
from conf import model_config_bert as model_config
class LinearEmbed(nn.Module):
    """Embedding module"""
    def __init__(self, dim_in=128, dim_out=2):
        super(LinearEmbed, self).__init__()
        self.linear = nn.Linear(dim_in, dim_out)
        # self.l2norm = Normalize(2)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        # x = self.l2norm(x)
        return x
tokenizer = XLNetTokenizer.from_pretrained("data/pretrain_model/twitter/xlnet-base-cased")
model_text = XLNetModel.from_pretrained("data/pretrain_model/twitter/xlnet-base-cased")
text_classifer=LinearEmbed(15360,2)
model_text.to('cuda')
text_classifer.to('cuda')
def text_embedding(text, text_model_path=None):
    emb_set=[]
    sentences = nltk.sent_tokenize(text)
    if text_model_path:
        
        checkpoint_text = torch.load(text_model_path)
        model_text.load_state_dict(checkpoint_text['model'])
    for sent in sentences:
        input_ids = torch.tensor(tokenizer.encode(sent)).unsqueeze(0)
        input_ids = input_ids.to('cuda')
        outputs= model_text(input_ids)
        cls_output = torch.mean(outputs[0], axis=1)
        emb_set.append(cls_output)
        if len(emb_set)>=20:
            emb_set=emb_set[:20]
        else:
            deficit=20-len(emb_set)
            for i in range(deficit):
                emb_set.append(torch.zeros((1,768)).cuda())
    emb_set = torch.cat(emb_set, dim=0)
    emb_set= torch.flatten(emb_set,0).unsqueeze(0)
    text_logist=text_classifer(emb_set)
    return emb_set,text_logist
        
    
    

In [7]:
tex,_,_=train_dataset[0]
emb,logist=text_embedding(tex)
print(emb.size(),logist.size())




torch.Size([1, 15360]) torch.Size([1, 2])


In [8]:
from PIL import Image
from PIL import ImageFile
import torchvision.transforms as transforms
from model.resnet import resnet50
transform_train = transforms.Compose([     
#             transforms.Resize((256, 128), interpolation=3),
        transforms.Resize((224,224), interpolation=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
transform_test = transforms.Compose([     
        transforms.Resize((224,224), interpolation=3),  # Image.BICUBIC
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
model_img = resnet50(pretrained=True,num_classes=2)
model_img.to('cuda')
def img_embedding(img_name,is_train, img_model_path=None):
    if is_train:
        img_path= 'AAAI_dataset/Images/politi_train/'+img_name
        img = Image.open(img_path).convert('RGB')   
        res_img=transform_train(img).float().unsqueeze(0).float()
    else:
        img_path= 'AAAI_dataset/Images/politi_test/'+img_name
        img = Image.open(img_path).convert('RGB')   
        res_img=transform_test(img).float().unsqueeze(0).float()   
    if img_model_path:
        checkpoint = torch.load(img_model_path)
        model_img.load_state_dict(checkpoint['model'])
    res_img=res_img.to('cuda')
    img_feat,img_logist=model_img(res_img)
    return img_feat,img_logist


        

In [9]:
_,train_img,_=train_dataset[0]
_,test_img,_=test_dataset[0]
train_img_feat, train_img_logist=img_embedding(train_img,is_train=True)
test_img_feat, test_img_logist=img_embedding(test_img,is_train=False)
train_img_feat.size(), train_img_logist.size(), test_img_feat.size(), test_img_logist.size()

(torch.Size([1, 2048]),
 torch.Size([1, 2]),
 torch.Size([1, 2048]),
 torch.Size([1, 2]))

In [10]:

from crd.criterion import CRDLoss
import torch.optim as optim
criterion_cls = nn.CrossEntropyLoss()
criterion_crd = CRDLoss(feat_dim=128, n_data=len(train_dataset), nce_k=250, nce_t=0.07, nce_m=0.5)
criterion_cls=criterion_cls.cuda()
criterion_crd=criterion_crd.cuda()
optimizer_img = optim.SGD(model_img.parameters(), lr=0.0003, momentum=0.9, weight_decay=5e-4)
optimizer_text = torch.optim.Adam(model_text.parameters(), lr=2e-5)   #1e-5, 2e-5, 3e-5

module_list = nn.ModuleList([])
module_list.append(criterion_crd.embed_s)
module_list.append(criterion_crd.embed_t)
module_list.append(model_img)
module_list.append(model_text)
module_list.append(text_classifer)


ModuleList(
  (0): Embed(
    (linear): Linear(in_features=2048, out_features=128, bias=True)
    (l2norm): Normalize()
  )
  (1): Embed(
    (linear): Linear(in_features=15360, out_features=128, bias=True)
    (l2norm): Normalize()
  )
  (2): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, 

In [11]:
import torch
import random
import copy
import numpy as np


def generate_embedding_batch(k, batch_data, is_train, img_model_path=None, text_model_path=None ):
    batch_text_emb,batch_img_emb, batch_label=[],[],[]
    batch_text_logist,batch_img_logist=[],[]
    batch_index, batch_contrast_idx=[],[]
    for idx, data in enumerate(batch_data):
        text, img_name, label=data
        text_item_emb,text_item_logist = text_embedding(text, text_model_path)
        img_item_emb,img_item_logist = img_embedding(img_name,is_train=is_train, img_model_path=img_model_path)
        batch_text_emb.append(text_item_emb)
        batch_img_emb.append(img_item_emb)
        batch_text_logist.append(text_item_logist)
        batch_img_logist.append(img_item_logist)
        batch_label.append(label)
        if is_train:
            pos_idx=train_dataset.index(data)
            batch_index.append(pos_idx)
            if label==0:
                replace = True if k > len(label_set[1]) else False
                neg_idx = np.random.choice(label_set[1], k, replace=replace)
            else:
                replace = True if k > len(label_set[0]) else False
                neg_idx = np.random.choice(label_set[0], k, replace=replace)
            sample_idx = torch.tensor(np.hstack((np.asarray([pos_idx]), neg_idx)))  
            batch_contrast_idx.append(sample_idx)

    batch_text_emb=torch.cat(batch_text_emb,0)
    batch_img_emb=torch.cat(batch_img_emb,0)
    batch_text_logist=torch.cat(batch_text_logist,0)
    batch_img_logist=torch.cat(batch_img_logist,0)
    if is_train:
        batch_index=torch.tensor(batch_index)
        batch_contrast_idx=torch.stack(batch_contrast_idx,0)
    batch_label=torch.tensor(batch_label)
    if is_train:
        return batch_text_emb,batch_img_emb, batch_text_logist, batch_img_logist, batch_index, batch_contrast_idx,batch_label
    else:
        return batch_text_emb,batch_img_emb, batch_text_logist, batch_img_logist, batch_label



    
        
        
        
        
    
        


In [12]:
# train_set=copy.deepcopy(train_dataset)    
# test_batch=train_set[:16]
# batch_text_emb,batch_img_emb, batch_text_logist, batch_img_logist, batch_index, batch_contrast_idx, batch_label=generate_embedding_batch(64,test_batch,is_train=True)
# print(batch_text_emb.size())
# print(batch_img_emb.size())
# print(batch_text_logist.size())
# print(batch_img_logist.size())
# print(batch_index)
# print(batch_contrast_idx)
# print(batch_label)

In [13]:
from helper import *

def train_epoch(epoch, train_set):
    for module in module_list:
        module.train()
        module.cuda()
    losses_img = AverageMeter()
    losses_text = AverageMeter()
    top1_img = AverageMeter()
    top1_text = AverageMeter() 
    cur_step=0
    for i in range(0,384,16):  
        cur_step+=1
        if i<368:
            batch_data=train_set[i:i+16]
        else:
            batch_data=train_set[i:]
        batch_text_emb,batch_img_emb, batch_text_logist, \
        batch_img_logist, batch_index, batch_contrast_idx, batch_label = generate_embedding_batch(64,batch_data, is_train=True)
        
        batch_index=batch_index.cuda()
        batch_contrast_idx=batch_contrast_idx.cuda()
        batch_label=batch_label.cuda()
        
        loss_cls_img = criterion_cls(batch_img_logist, batch_label)
        loss_cls_text = criterion_cls(batch_text_logist, batch_label)

        loss_kd = criterion_crd(batch_img_emb, batch_text_emb, batch_index, batch_contrast_idx)

        train_loss_img =loss_cls_img + 0.3 * loss_kd
        train_loss_text = loss_cls_text  + 0.3 * loss_kd

        acc1_img = accuracy(batch_img_logist, batch_label)
        losses_img.update(train_loss_img.item(), batch_index.size(0))
        top1_img.update(float(acc1_img[0]), batch_index.size(0))

        optimizer_img.zero_grad()
        train_loss_img.backward(retain_graph=True)
        optimizer_img.step()

        acc1_text = accuracy(batch_text_logist, batch_label)
        losses_text.update(train_loss_text.item(), batch_index.size(0))
        top1_text.update(float(acc1_text[0]), batch_index.size(0))

        optimizer_text.zero_grad()
        train_loss_text.backward(retain_graph=True)
        optimizer_text.step()
        if cur_step % 3 == 0:
            print('Train for Image Model:')
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                epoch,
                loss=losses_img, top1=top1_img))

            print('Train for Text Model:')
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                epoch, 
                loss=losses_text, top1=top1_text))
            print('\n')
    print(' Train image model : Acc@1 {top1.avg:.3f}'.format(top1=top1_img))
    print(' Train text model : Acc@1 {top1.avg:.3f}'.format(top1=top1_text))
        

In [14]:
from helper import *

def test_epoch(epoch, test_set):
    for module in module_list:
        module.eval()
        module.cuda()
    losses_img = AverageMeter()
    top1_img = AverageMeter()
    losses_text = AverageMeter()
    top1_text = AverageMeter()
    cur_step=0
    with torch.no_grad():
        for i in range(0,112,16):  
            cur_step += 1
            if i<96:
                batch_data=test_set[i:i+16]
            else:
                batch_data=test_set[i:]
            batch_text_emb,batch_img_emb, batch_text_logist, batch_img_logist, batch_label = generate_embedding_batch(64,batch_data, is_train=False)
            batch_label=batch_label.cuda()

            loss_cls_img = criterion_cls(batch_img_logist, batch_label)
            loss_cls_text = criterion_cls(batch_text_logist, batch_label)



            acc1_img = accuracy(batch_img_logist, batch_label)
            losses_img.update(loss_cls_img.item(), batch_label.size(0))
            top1_img.update(float(acc1_img[0]), batch_label.size(0))

            

            acc1_text = accuracy(batch_text_logist, batch_label)
            losses_text.update(loss_cls_text.item(), batch_label.size(0))
            top1_text.update(float(acc1_text[0]), batch_label.size(0))

            
            if cur_step%3 == 0:

                print('Test for Image Model:')
                print(
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(

                    loss=losses_img, top1=top1_img))


                print('Test for Text Model:')
                print(
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(

                    loss=losses_text, top1=top1_text))
                print('\n')
        
        print('Test for img model * Acc@1 {top1.avg:.3f} '.format(top1=top1_img))
        print('Test for text model : Acc@1 {top1.avg:.3f}'.format(top1=top1_text))
        print('\n')
        return top1_img.avg, top1_text.avg
        

In [15]:
from conf import config
best_img_acc = 0
best_text_acc = 0
img_save_file = None
text_save_file = None
train_set=copy.deepcopy(train_dataset)
test_set=copy.deepcopy(test_dataset)
sys.stdout = Logger(os.path.join(config.save_folder, 'log.txt'))
for epoch_id in range(30):
    print('Epoch:', epoch_id)
    random.shuffle(train_set)
    train_epoch(epoch_id, train_set)
    test_img_acc,test_text_acc = test_epoch(epoch_id,test_set)
    if test_img_acc > best_img_acc:
        best_img_acc = test_img_acc
        state = {
            'epoch': epoch_id,
            'model': model_img.state_dict(),
            'best_acc': best_img_acc,
        }
        img_save_file = os.path.join(config.save_folder, 'img_best.pth')
        print('saving the best model for img!')
        torch.save(state, img_save_file)
    if test_text_acc > best_text_acc:
        best_text_acc = test_text_acc
        state = {
            'epoch': epoch_id,
            'model': model_text.state_dict(),
            'best_acc': best_text_acc,
        }
        text_save_file = os.path.join(config.save_folder, 'text_best.pth')
        print('saving the best model for text!')
        torch.save(state, text_save_file)
        print('\n')
print('best accuracy for image model :', best_img_acc)
print('best accuracy for text model :', best_text_acc)

Epoch: 0


RuntimeError: shape '[16, 251, 128]' is invalid for input of size 133120

In [ ]:
# fc = LinearEmbed(17408, 2) 
# optimizer_fc = torch.optim.Adam(fc.parameters(), lr=2e-5)
# img_model_path='data/log/notebook_politi/img_best.pth'
# text_model_path='data/log/notebook_politi/text_best.pth'


In [ ]:
def train_fc_epoch(epoch, train_set):
    for module in module_list:
        module.eval()
        module.cuda()
    fc.train()
    fc.cuda()
    
    losses_fc = AverageMeter()
    top1_fc = AverageMeter()
    cur_step=0
    
    for i in range(0,384,16):  
        cur_step+=1
        if i<368:
            batch_data=train_set[i:i+16]
        else:
            batch_data=train_set[i:]
        with torch.no_grad():
            batch_text_emb,batch_img_emb, _, \
            _, _, _, batch_label = generate_embedding_batch(64,batch_data, is_train=True,img_model_path=img_model_path, text_model_path=text_model_path) 
            batch_label=batch_label.cuda()
            batch_text_emb=batch_text_emb.detach()
            batch_img_emb=batch_img_emb.detach()
        multi_feature = torch.cat((batch_text_emb, batch_img_emb), 1)
        logist_fc= fc(multi_feature)
        
        loss_fc = criterion_cls(logist_fc, batch_label)
        acc1_fc = accuracy(logist_fc, batch_label)
        
        losses_fc.update(loss_fc.item(), batch_label.size(0))
        top1_fc.update(float(acc1_fc[0]), batch_label.size(0))

        optimizer_fc.zero_grad()
        loss_fc.backward()
        optimizer_fc.step()

        if cur_step%3 == 0:
            print('Train for Fc:')
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                epoch,
                loss=losses_fc, top1=top1_fc))
            print('\n')
    print(' Train Fc : Acc@1 {top1.avg:.3f}'.format(top1=top1_fc))
    
    

In [ ]:
def test_fc_epoch(epoch, test_set):
    for module in module_list:
        module.eval()
        module.cuda()
    fc.eval()
    fc.cuda()
    
    losses_fc = AverageMeter()
    top1_fc = AverageMeter()
    count_0_lists = []
    correct_0_lists = []
    target_0_lists = []
    count_1_lists = []
    correct_1_lists = []
    target_1_lists = []
    cur_step=0
    
    with torch.no_grad():
        for i in range(0,112,16):  
            cur_step += 1
            if i<96:
                batch_data=test_set[i:i+16]
            else:
                batch_data=test_set[i:]
    
            batch_text_emb,batch_img_emb, _, _, batch_label = \
        generate_embedding_batch(64,batch_data,is_train=False,img_model_path=img_model_path, text_model_path=text_model_path) 
            batch_label=batch_label.cuda()
            batch_text_emb=batch_text_emb.detach()
            batch_img_emb=batch_img_emb.detach()
            multi_feature = torch.cat((batch_text_emb, batch_img_emb), 1).detach()
            logist_fc= fc(multi_feature)

            loss_fc = criterion_cls(logist_fc, batch_label)
            acc1_fc = accuracy(logist_fc, batch_label)
            
            count_0, count_correct_0, count_target_0 = metric(logist_fc, batch_label, for_fake=True)
            count_1, count_correct_1, count_target_1 = metric(logist_fc, batch_label, for_fake=False)
            count_0_lists.append(count_0)
            correct_0_lists.append(count_correct_0)
            target_0_lists.append(count_target_0)

            count_1_lists.append(count_1)
            correct_1_lists.append(count_correct_1)
            target_1_lists.append(count_target_1)

            losses_fc.update(loss_fc.item(), batch_label.size(0))
            top1_fc.update(float(acc1_fc[0]), batch_label.size(0))

            if cur_step % 3 == 0:
                print('Test for Fc:')
                print('Epoch: [{0}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                    epoch,
                    loss=losses_fc, top1=top1_fc))
                print('\n')
        epoch_count_0 = sum(count_0_lists)
        epoch_correct_0 = sum(correct_0_lists)
        epoch_target_0 = sum(target_0_lists)

        epoch_count_1 = sum(count_1_lists)
        epoch_correct_1 = sum(correct_1_lists)
        epoch_target_1 = sum(target_1_lists)

        try:
            prec_0 = float(epoch_correct_0) * (100.0 / float(epoch_count_0))
        except ZeroDivisionError:
            prec_0 = 0
        try:
            rec_0 = float(epoch_correct_0) * (100.0 / float(epoch_target_0))
        except ZeroDivisionError:
            rec_0 = 0
        try:
            f_0 = 2 * (prec_0 * rec_0) / (prec_0 + rec_0)
        except ZeroDivisionError:
            f_0 = 0

        try:
            prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        except ZeroDivisionError:
            prec_1 = 0
        try:
            rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        except ZeroDivisionError:
            rec_1 = 0
        try:
            f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        except ZeroDivisionError:
            f_1 = 0

        # prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        # rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        # f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        print('val * Acc@1 {top1.avg:.3f} '.format(top1=top1_fc))
        print('metrics: prec_0, rec_0, f_0, prec_1, rec_1, f_1')
        print(prec_0, rec_0, f_0, prec_1, rec_1, f_1)

    return top1_fc.avg, prec_0, rec_0, f_0, prec_1, rec_1, f_1
        

In [ ]:
# best_acc=0
# fc_save_file = None

# best_prec_0, best_rec_0, best_f_0 = 0, 0, 0
# best_prec_1, best_rec_1, best_f_1 = 0, 0, 0
# full_metric = None


# train_set=copy.deepcopy(train_dataset)
# test_set=copy.deepcopy(test_dataset)




# sys.stdout = Logger(os.path.join(config.save_folder, 'fc.txt'))
# for epoch_id in range(60):
#     print('Epoch:', epoch_id)
#     random.shuffle(train_set)
#     train_fc_epoch(epoch_id, train_set)
#     acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1 = test_fc_epoch(epoch_id,test_set)
#     if acc > best_acc:
#         best_acc = acc
#         full_metric = (acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1)
#         print('best acc in epoch:{}'.format(epoch_id), best_acc)
#         print('full metric when best acc: prec_0, rec_0, f_0, prec_1, rec_1, f_1')
#         print(full_metric)
#         print('\n')
#     if prec_0 > best_prec_0:
#         best_prec_0 = prec_0
#         print('best prec_0 in epoch {}:'.format(epoch_id), best_prec_0)
#     if rec_0 > best_rec_0:
#         best_rec_0 = rec_0
#         print('best rec_0 in epoch {}:'.format(epoch_id), best_rec_0)
#     if f_0 > best_f_0:
#         best_f_0 = f_0
#         print('best f_0 in epoch {}:'.format(epoch_id), best_f_0)

#     if prec_1 > best_prec_1:
#         best_prec_1 = prec_1
#         print('best prec_1 in epoch {}:'.format(epoch_id), best_prec_1)
#     if rec_1 > best_rec_1:
#         best_rec_1 = rec_1
#         print('best rec_1 in epoch {}:'.format(epoch_id), best_rec_1)
#     if f_1 > best_f_1:
#         best_f_1 = f_1
#         print('best f_1 in epoch {}:'.format(epoch_id), best_f_1)

# print('best accuracy :', best_acc)
# print('full metric under best accuracy: prec, rec, F1, prec_fake, rec_fake, F1_fake')
# print(full_metric)

# print('best prec_0 :', best_prec_0)
# print('best rec_0 :', best_rec_0)
# print('best f_0 :', best_f_0)
# print('best prec_1 :', best_prec_1)
# print('best rec_1 :', best_rec_1)
# print('best f_1 :', best_f_1)
#     if test_fc_acc > best_fc_acc:
#         best_fc_acc = test_fc_acc
#         state = {
#             'epoch': epoch_id,
#             'model': fc.state_dict(),
#             'best_acc': best_fc_acc,
#         }
#         fc_save_file = os.path.join(config.save_folder, 'fc.pth')
#         print('saving the best model for fc!')
#         torch.save(state, fc_save_file)
#         print('\n')
# print('best accuracy for fc model :', best_fc_acc)

In [ ]:
from model.block import Block
block = Block([2048, 15360], 2)
img_model_path='data/log/xlnet-base-cased/politi/debate_k/notebook/64/img_best.pth'
text_model_path='data/log/xlnet-base-cased/politi/debate_k/notebook/64/text_best.pth'
optimizer_block = torch.optim.Adam(block.parameters(), lr=2e-5)


In [ ]:
def train_block_epoch(epoch, train_set):
    for module in module_list:
        module.eval()
        module.cuda()
    block.train()
    block.cuda()
    
    losses_block = AverageMeter()
    top1_block = AverageMeter()
    cur_step=0
    
    for i in range(0,384,16):  
        cur_step+=1
        if i<368:
            batch_data=train_set[i:i+16]
        else:
            batch_data=train_set[i:]
        with torch.no_grad():
            batch_text_emb,batch_img_emb, _, \
            _, _, _, batch_label = generate_embedding_batch(64,batch_data, is_train=True,img_model_path=img_model_path, text_model_path=text_model_path) 
            batch_label=batch_label.cuda()
            batch_text_emb=batch_text_emb.detach()
            batch_img_emb=batch_img_emb.detach()
        block_output = block([batch_img_emb,batch_text_emb])
        
        
        loss_block = criterion_cls(block_output, batch_label)
        acc1_block = accuracy(block_output, batch_label)
        
        losses_block.update(loss_block.item(), batch_label.size(0))
        top1_block.update(float(acc1_block[0]), batch_label.size(0))

        optimizer_block.zero_grad()
        loss_block.backward()
        optimizer_block.step()

        if cur_step%3 == 0:
            print('Train for Block:')
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                epoch,
                loss=losses_block, top1=top1_block))
            print('\n')
    print(' Train Block : Acc@1 {top1.avg:.3f}'.format(top1=top1_block))
    
    

In [ ]:
def test_block_epoch(epoch, test_set):
    for module in module_list:
        module.eval()
        module.cuda()
    block.eval()
    block.cuda()
    
    losses_block = AverageMeter()
    top1_block = AverageMeter()
    count_0_lists = []
    correct_0_lists = []
    target_0_lists = []
    count_1_lists = []
    correct_1_lists = []
    target_1_lists = []
    cur_step=0
    
    with torch.no_grad():
        for i in range(0,112,16):  
            cur_step += 1
            if i<96:
                batch_data=test_set[i:i+16]
            else:
                batch_data=test_set[i:]
    
            batch_text_emb,batch_img_emb, _, _, batch_label = \
        generate_embedding_batch(64,batch_data,is_train=False,img_model_path=img_model_path, text_model_path=text_model_path) 
            batch_label=batch_label.cuda()
            batch_text_emb=batch_text_emb.detach()
            batch_img_emb=batch_img_emb.detach()
            block_output = block([batch_img_emb,batch_text_emb])
            
            loss_block = criterion_cls(block_output, batch_label)
            acc1_block = accuracy(block_output, batch_label)
            
            count_0, count_correct_0, count_target_0 = metric(block_output, batch_label, for_fake=True)
            count_1, count_correct_1, count_target_1 = metric(block_output, batch_label, for_fake=False)
            count_0_lists.append(count_0)
            correct_0_lists.append(count_correct_0)
            target_0_lists.append(count_target_0)

            count_1_lists.append(count_1)
            correct_1_lists.append(count_correct_1)
            target_1_lists.append(count_target_1)

            losses_block.update(loss_block.item(), batch_label.size(0))
            top1_block.update(float(acc1_block[0]), batch_label.size(0))

            if cur_step % 3 == 0:
                print('Test for Block:')
                print('Epoch: [{0}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                    epoch,
                    loss=losses_block, top1=top1_block))
                print('\n')
        epoch_count_0 = sum(count_0_lists)
        epoch_correct_0 = sum(correct_0_lists)
        epoch_target_0 = sum(target_0_lists)

        epoch_count_1 = sum(count_1_lists)
        epoch_correct_1 = sum(correct_1_lists)
        epoch_target_1 = sum(target_1_lists)

        try:
            prec_0 = float(epoch_correct_0) * (100.0 / float(epoch_count_0))
        except ZeroDivisionError:
            prec_0 = 0
        try:
            rec_0 = float(epoch_correct_0) * (100.0 / float(epoch_target_0))
        except ZeroDivisionError:
            rec_0 = 0
        try:
            f_0 = 2 * (prec_0 * rec_0) / (prec_0 + rec_0)
        except ZeroDivisionError:
            f_0 = 0

        try:
            prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        except ZeroDivisionError:
            prec_1 = 0
        try:
            rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        except ZeroDivisionError:
            rec_1 = 0
        try:
            f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        except ZeroDivisionError:
            f_1 = 0

        # prec_1 = float(epoch_correct_1) * (100.0 / float(epoch_count_1))
        # rec_1 = float(epoch_correct_1) * (100.0 / float(epoch_target_1))
        # f_1 = 2 * (prec_1 * rec_1) / (prec_1 + rec_1)
        print('val * Acc@1 {top1.avg:.3f} '.format(top1=top1_block))
        print('metrics: prec_0, rec_0, f_0, prec_1, rec_1, f_1')
        print(prec_0, rec_0, f_0, prec_1, rec_1, f_1)

    return top1_block.avg, prec_0, rec_0, f_0, prec_1, rec_1, f_1
        
#         print(' Test Block : Acc@1 {top1.avg:.3f}'.format(top1=top1_block))
#     return top1_block.avg

In [ ]:
# best_acc = 0
# block_save_file = None
# best_prec_0, best_rec_0, best_f_0 = 0, 0, 0
# best_prec_1, best_rec_1, best_f_1 = 0, 0, 0
# full_metric = None


# train_set=copy.deepcopy(train_dataset)
# test_set=copy.deepcopy(test_dataset)
# sys.stdout = Logger(os.path.join(config.save_folder, 'block.txt'))
# for epoch_id in range(30):
#     print('Epoch:', epoch_id)
#     random.shuffle(train_set)
#     train_block_epoch(epoch_id, train_set)
#     acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1  = test_block_epoch(epoch_id,test_set)
#     if acc > best_acc:
#         best_acc = acc
#         full_metric = (acc, prec_0, rec_0, f_0, prec_1, rec_1, f_1)
#         print('best acc in epoch:{}'.format(epoch_id), best_acc)
#         print('full metric when best acc: prec_0, rec_0, f_0, prec_1, rec_1, f_1')
#         print(full_metric)
#         print('\n')
#     if prec_0 > best_prec_0:
#         best_prec_0 = prec_0
#         print('best prec_0 in epoch {}:'.format(epoch_id), best_prec_0)
#     if rec_0 > best_rec_0:
#         best_rec_0 = rec_0
#         print('best rec_0 in epoch {}:'.format(epoch_id), best_rec_0)
#     if f_0 > best_f_0:
#         best_f_0 = f_0
#         print('best f_0 in epoch {}:'.format(epoch_id), best_f_0)

#     if prec_1 > best_prec_1:
#         best_prec_1 = prec_1
#         print('best prec_1 in epoch {}:'.format(epoch_id), best_prec_1)
#     if rec_1 > best_rec_1:
#         best_rec_1 = rec_1
#         print('best rec_1 in epoch {}:'.format(epoch_id), best_rec_1)
#     if f_1 > best_f_1:
#         best_f_1 = f_1
#         print('best f_1 in epoch {}:'.format(epoch_id), best_f_1)

# print('best accuracy :', best_acc)
# print('full metric under best accuracy: prec, rec, F1, prec_fake, rec_fake, F1_fake')
# print(full_metric)

# print('best prec_0 :', best_prec_0)
# print('best rec_0 :', best_rec_0)
# print('best f_0 :', best_f_0)
# print('best prec_1 :', best_prec_1)
# print('best rec_1 :', best_rec_1)
# print('best f_1 :', best_f_1)
#     if test_block_acc > best_block_acc:
#         best_block_acc = test_block_acc
#         state = {
#             'epoch': epoch_id,
#             'model': block.state_dict(),
#             'best_acc': best_block_acc,
#         }
#         block_save_file = os.path.join(config.save_folder, 'block.pth')
#         print('saving the best model for block!')
#         torch.save(state, block_save_file)
#         print('\n')
# print('best accuracy for block model :', best_block_acc)